## Web scraping From Reddit API

### Installing and Importing Some libraries

In [1]:
!pip install praw
!pip install nltk
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [2]:
import praw
import pandas as pd 
import nltk
import re
from datetime import datetime

### Creating a dataframe for some reddit posts with subreddit names

In [3]:
df = pd.DataFrame(columns=['title','author','date','score','Num_coments'])

In [4]:
reddit = praw.Reddit(client_id="sCiuOpj_dnS7AQ",
                       client_secret="ORRVwLlngtNPE0sWcWwvvGWaiT-iEg",
                       user_agent="I am",
                       password="YNRana16#coder",
                       username="YNRana")

In [5]:
testing1 = list(reddit.subreddit("investing+wallstreetbets").new(limit = 100000))
testing2 = list(reddit.subreddit("wallstreet+SecurityAnalysis").new(limit = 100000))
testing3 = list(reddit.subreddit("StockMarket+pennystocks+algotrading").new(limit = 100000))
testing = testing1+testing2+testing3
print(len(testing1),len(testing2),len(testing3))
for index in range(1,len(testing)):
#     print(post.title,"\t", post.author)
    data = int(testing[index].created_utc)
    date = datetime.utcfromtimestamp(data).strftime('%Y-%m-%d %H:%M:%S')
    df.loc[index]= [str(testing[index].title)] + [str(testing[index].author)] + [str(date)] + [str(testing[index].score)] + [str(testing[index].num_comments)]
                                                     
df.tail()    

,title,author,date,score,Num_coments
6065,Issues with Yahoo Finance API for SPX historic...,DavidWeirich,2021-03-23 22:12:53,2,0
6066,Forward Testing Only????,zenonu,2021-03-23 20:15:29,4,3
6067,Anti-RSI Challenge,SethEllis,2021-03-23 18:06:35,6,2
6068,What are the uses of a Websocket for an exchan...,CyberPunkMetalHead,2021-03-23 18:04:46,18,17
6069,MQL5 (or 4): waste of time?,quadtodfodder,2021-03-23 17:43:19,5,8


### Extracting Comapany name from the posts

In [6]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [7]:
from nltk import word_tokenize,pos_tag
from nltk import Tree
def get_continuous_chunks(text, label):
    print(text)
    data_list = []
    for token_word in word_tokenize(text):
        real_token = token_word
        token_word = token_word.lower()
        for token,pos in pos_tag([token_word]):
            print(token,pos)
            if str(pos) =='NN' or pos.startswith('NNP'):
                word = real_token
            else:
                word = token_word
        data_list.append(word)

    return data_list

In [8]:
title_list = []
for index in range(1,len(df)):
    lis1 = get_continuous_chunks(df.iloc[index]['title'], 'ORGANIZATION')
    str1 = ' '.join(lis1)
    title_list.append([str1,df.iloc[index]['date'],df.iloc[index]['score'],df.iloc[index]['Num_coments']])
print(title_list)   

Streaming output truncated to the last 5000 lines.
stop NN
for IN
short JJ
position NN
Fourier and Bernoulli
fourier NN
and CC
bernoulli NN
Genetic algorithms vs random parameter optimization
genetic JJ
algorithms NN
vs NN
random NN
parameter NN
optimization NN
Resources to identify support and resistance lines programmatically (python).
resources NNS
to TO
identify VB
support NN
and CC
resistance NN
lines NNS
programmatically RB
( (
python NN
) )
. .
Workflow for strategy development to production
workflow NN
for IN
strategy NN
development NN
to TO
production NN
Quant developer believes all future prices are random and cannot be predicted
quant NN
developer NN
believes VBZ
all DT
future NN
prices NNS
are VBP
random NN
and CC
can MD
not RB
be VB
predicted VBN
Best way to monitor scripts remotely...?
best JJS
way NN
to TO
monitor NN
scripts NNS
remotely RB
... :
? .
Detecting meta-orders in Limit Order Book markets
detecting VBG
meta-orders NNS
in IN
limit NN
order NN
book NN
markets NN

In [9]:
import spacy 
from spacy import displacy
#SpaCy 2.x brough significant speed and accuracy improvements
spacy.__version__

'2.2.4'

In [10]:
nlp = spacy.load("en_core_web_sm")

In [11]:
name = []
score = []
Num_comments = []
position_end = []
title = []
date = []
for sent in title_list:
    doc = nlp(sent[0])

    

    for ent in doc.ents:
        if ent.label_ =='ORG':
            name.append(str(ent))
            title.append(sent[0])
            date.append(sent[1])
            score.append(sent[2])
            Num_comments.append(sent[3])

dfs = pd.DataFrame({'title':title,'name':name,'date':date,"score":score,"Num_comments":Num_comments})
dfs

,title,name,date,score,Num_comments
0,YOLO CLOV all in with AUNT 's may RETIREMENT F...,RUSSEL 2000,2021-05-29 11:28:06,1,9
1,YOLO CLOV all in with AUNT 's may RETIREMENT F...,HIDENBURG,2021-05-29 11:28:06,1,9
2,YOLO CLOV all in with AUNT 's may RETIREMENT F...,BAC,2021-05-29 11:28:06,1,9
3,YOLO CLOV all in with AUNT 's may RETIREMENT F...,MOON,2021-05-29 11:28:06,1,9
4,is it possible to invest in or short - either ...,Mortgage,2021-05-29 09:12:49,7,9
...,...,...,...,...,...
2908,questions about TDAmeritrade options API,TDAmeritrade,2021-03-24 05:54:57,1,0
2909,is there a good simple tutorial for the intera...,API,2021-03-24 00:35:10,31,14
2910,issues with Yahoo Finance API for SPX historic...,Yahoo Finance API,2021-03-23 22:12:53,2,0
2911,issues with Yahoo Finance API for SPX historic...,SPX,2021-03-23 22:12:53,2,0


### Removing Duplicate rows and downloading the DataFrame as csv file

In [12]:
newrecord = dfs.drop_duplicates(['title'])
newrecord

,title,name,date,score,Num_comments
0,YOLO CLOV all in with AUNT 's may RETIREMENT F...,RUSSEL 2000,2021-05-29 11:28:06,1,9
4,is it possible to invest in or short - either ...,Mortgage,2021-05-29 09:12:49,7,9
5,daily general Discussion and spitballin thread...,Discussion,2021-05-29 09:01:20,1,1
6,is there a way to tell how much or how many sh...,etf,2021-05-29 08:02:29,0,2
7,WANT to RETIRE QUICKER ? here 's how !,RETIRE QUICKER,2021-05-29 07:37:06,39,50
...,...,...,...,...,...
2907,best financial data API ?,API,2021-03-24 13:08:04,1,1
2908,questions about TDAmeritrade options API,TDAmeritrade,2021-03-24 05:54:57,1,0
2909,is there a good simple tutorial for the intera...,API,2021-03-24 00:35:10,31,14
2910,issues with Yahoo Finance API for SPX historic...,Yahoo Finance API,2021-03-23 22:12:53,2,0


In [13]:
newrecord.to_csv('final1.csv')